# Deploying ML Models

In [ ]:
                                        Deploying ML Models
    
    Web hosting frameworks                HTTP endpoints                Cloud Functions for specific
                                                                                frameworks
    
    - Flask                              Severless compute                     - SageMaker
    - Django                            - AWS Lambda                           - Google AI Platform
                                        - Google Cloud Functions               - Azure ML Service
                                        - Azure Functions

In [ ]:
compute(Software Architect)

Bare Metal > VM Instances >  Container Clusters > Hosted Apps > Serverless Functions
             (light weight)   (to isolate apps)
    
    -> less operational(admin)
    <- more control, low level access

In [1]:
import sklearn
import pickle
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('sentiment_analysis.csv')
df.head()

,text,label
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [4]:
df.shape

(7100, 2)

In [5]:
x = df['text']

y = df['label']

xtrain, xtest, ytrain, ytest = train_test_split(x,y, test_size=0.2)

In [6]:
tfidf_vector = TfidfVectorizer(max_features=15)

In [7]:
logistic_classifier = LogisticRegression(solver='liblinear')

In [8]:
classification_pipeline = Pipeline(steps=[('tfidf_vector', tfidf_vector),
                                          ('classifier', logistic_classifier)])

pipeline_model = classification_pipeline.fit(xtrain, ytrain)

ypred = pipeline_model.predict(xtest)

accuracy_score(ytest, ypred)

0.5964788732394366

In [9]:
# serialization

pickle.dump(pipeline_model, open('models/logistic_classifier/logistic_model.pkl', 'wb'))

In [10]:
# decision tree classifier

decision_tree_classifier = DecisionTreeClassifier(max_depth=10)

classification_pipeline = Pipeline(steps=[('tfidf_vector', tfidf_vector),
                                          ('classifier', decision_tree_classifier)])

pipeline_model = classification_pipeline.fit(xtrain, ytrain)

ypred = pipeline_model.predict(xtest)

accuracy_score(ytest, ypred)

0.5845070422535211

In [11]:
pickle.dump(pipeline_model, open('models/decision_classifier/decision_model.pkl', 'wb'))

In [12]:
# linear SVC

linear_svc = LinearSVC(C=1.0, max_iter=100)

classification_pipeline = Pipeline(steps=[('tfidf_vector', tfidf_vector),
                                          ('classifier', linear_svc)])

pipeline_model = classification_pipeline.fit(xtrain, ytrain)

ypred = pipeline_model.predict(xtest)

accuracy_score(ytest, ypred)

0.5964788732394366

In [13]:
pickle.dump(pipeline_model, open('models/support_classifier/support_model.pkl', 'wb'))

In [ ]:
- upload models to GCP
- cloud functions

In [14]:
!python --version

Python 3.9.12


In [19]:
!sklearn --version

'sklearn' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
events -> triggers -> serverless function -> invoke other services on cloud

events
- HTTP
- storage access
- chatbot interaction
- 

## cloud function

main.py

In [ ]:
import requests
import pickle
from google.cloud import storage

def classifier(request):
    if request.method == 'GET':
        return "welcome to Classifier"
    
    if request.method == 'POST':
        data = request.get_json()
        storage_client = storage.Client()
        bucket = storage_client.get_bucket("classifier-models")
        
        if data['model'] == ['Decisionclassifier']:
            blob = bucket.blob('models/decision_tree_model/model.pkl')
        elif data['model'] == ['LinearSVC']:
            blob = bucket.blob('models/linear_svc_model/model.pkl')
        else:
            blob = bucket.blob('models/logistic_regression_model/model.pkl')
        blob.download_to_filename('/tmp/model.pkl')
        model = pickle.load(open('/tmp/model.pkl', 'rb'))
        
        x_data = data['x']
        
        output = model.predict(x_data)
        pred_class = 'Text:' + str(x_data) + 'Predicted class is:' + str(output)
        
    return pred_class
        

In [20]:
# !pip freeze